# TrustCallJS - Getting Started

This notebook demonstrates how to use TrustCallJS for validated tool calling and extraction with retries using LLMs.

## Prerequisites

- Deno installed with Jupyter kernel (`deno jupyter --install`)
- OpenAI API key set as environment variable `OPENAI_API_KEY`

## Setup

First, let's import the required dependencies:

In [1]:
import { z } from "npm:zod";
import { ChatOpenAI } from "npm:@langchain/openai";

Import TrustCallJS (use the local build or npm package):

In [2]:
// For local development, import from the dist folder:
import { createExtractor, ValidationNode } from "../dist/trustcall/index.js";

// For npm package:
// import { createExtractor, ValidationNode } from "npm:trustcalljs";

## Example 1: Basic Schema Extraction

Extract structured user information from natural language text.

In [3]:
// Define a schema for user information
const UserInfo = z.object({
  name: z.string().describe("User's full name"),
  age: z.number().describe("User's age in years"),
  email: z.string().email().optional().describe("User's email address"),
}).describe("UserInfo");

console.log("Schema defined:", UserInfo.description);

Schema defined: UserInfo


In [4]:
// Initialize the LLM
const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0
});

console.log("LLM initialized");

LLM initialized


In [5]:
// Create the extractor
const extractor = createExtractor(llm, {
  tools: [UserInfo],
});

// // Extract structured data
const result = await extractor.invoke("My name is Alice Johnson and I'm 30 years old. You can reach me at alice@example.com");

console.log("Extracted data:", result.responses[0]);
console.log("Attempts needed:", result.attempts);

Extracted data: { name: "Alice Johnson", age: 30, email: "alice@example.com" }
Attempts needed: 1


## Example 2: Complex Nested Schema

TrustCallJS excels at handling complex, deeply nested schemas that often cause validation errors with standard approaches.

In [6]:
// Define a complex nested schema
const CompanyProfile = z.object({
  name: z.string().describe("Company name"),
  founded: z.number().describe("Year founded"),
  headquarters: z.object({
    city: z.string(),
    country: z.string(),
    address: z.string().optional(),
  }).describe("Company headquarters location"),
  leadership: z.array(z.object({
    name: z.string(),
    title: z.string(),
    since: z.number().optional(),
  })).describe("Company leadership team"),
  products: z.array(z.string()).describe("Main products or services"),
}).describe("CompanyProfile");

console.log("Complex schema defined:", CompanyProfile.description);

Complex schema defined: CompanyProfile


In [ ]:
// Create extractor for company profiles
import {HumanMessage} from "@langchain/core/messages"
const companyExtractor = createExtractor(llm, {
  tools: [CompanyProfile],
});

const companyResult = await companyExtractor.invoke({
  messages: new HumanMessage(`
    Anthropic is an AI safety company founded in 2021. 
    They're headquartered in San Francisco, USA.
    Dario Amodei is the CEO since founding, and Daniela Amodei is the President.
    Their main product is Claude, an AI assistant.
  `),
});

console.log("Extracted company profile:");
console.log(JSON.stringify(companyResult.responses[0], null, 2));

Error: Last message is not an AIMessage

## Example 3: Updating Existing Data

TrustCallJS can update existing schemas without losing information by using JSONPatch operations.

In [ ]:
// Define a user preferences schema
const UserPreferences = z.object({
  name: z.string(),
  favoriteColors: z.array(z.string()),
  settings: z.object({
    notifications: z.boolean(),
    theme: z.enum(["light", "dark", "system"]),
    language: z.string(),
  }),
}).describe("UserPreferences");

// Existing user data
const existingData = {
  UserPreferences: {
    name: "Alice",
    favoriteColors: ["blue", "green"],
    settings: {
      notifications: true,
      theme: "light",
      language: "en",
    },
  },
};

console.log("Existing data:", JSON.stringify(existingData, null, 2));

In [ ]:
// // Create extractor with updates enabled
// const updateExtractor = createExtractor(llm, {
//   tools: [UserPreferences],
//   enableUpdates: true,
// });

// // Update based on user request
// const updateResult = await updateExtractor.invoke({
//   messages: "I'd like to switch to dark theme and add purple to my favorite colors",
//   existing: existingData,
// });

// console.log("Updated data:");
// console.log(JSON.stringify(updateResult.responses[0], null, 2));
// // Note: name, notifications, language, and existing colors are preserved!

## Example 4: Using ValidationNode Directly

You can use the `ValidationNode` class directly in custom LangGraph workflows.

In [ ]:
// import { AIMessage } from "npm:@langchain/core/messages";
// import { ValidationNode } from "../dist/index.js";

// // Create a validation node
// const validator = new ValidationNode([UserInfo], {
//   formatError: (error, call, schema) => 
//     `Validation failed for ${call.name}: ${error.message}`,
// });

// // Simulate an AI message with tool calls
// const aiMessage = new AIMessage({
//   content: "Extracting user info",
//   tool_calls: [
//     {
//       id: "call-1",
//       name: "UserInfo",
//       args: { name: "Bob", age: 25 },
//     },
//   ],
// });

// // Validate the tool call
// const validationResult = await validator.invoke([aiMessage]);
// console.log("Validation result:", validationResult[0].content);

## How It Works

TrustCallJS uses a **patch-based extraction** approach:

1. **Initial Extraction**: The LLM generates tool calls based on input
2. **Validation**: Tool calls are validated against Zod schemas
3. **Error Detection**: Validation errors are detected and formatted
4. **Patch Generation**: The LLM generates JSONPatch operations to fix errors
5. **Application**: Patches are applied to the original arguments
6. **Retry**: Process repeats until validation passes or max attempts reached

This is more efficient than regenerating entire outputs because:
- Only failing parts are regenerated
- Existing correct data is preserved
- Fewer output tokens are needed for fixes

## Next Steps

- Check out more examples in the `examples/` directory
- Read the full API documentation
- Explore integration with LangGraph for custom workflows